In [1]:
from datetime import date,timedelta,datetime
from urllib.request import urlopen
from dateutil import rrule
import matplotlib.dates as mdates
import pandas as pd
import numpy as np
import json
import time

import requests
from io import StringIO
from dateutil.relativedelta import relativedelta
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

import os
os.makedirs("./Data",exist_ok=True)
from pathlib import Path

fle = Path('BaseInfo/His_Stock.csv')
if fle.exists()==False :
    aa=pd.DataFrame({
        "type":"",
        "stock_number":str(),
        "date":"",
        "status":"",
    },index=[0]) 
    aa.to_csv('BaseInfo/His_Stock.csv',index=False)

'台泥-水泥工業-上市'

# 檔案存取相關

In [2]:
traget_floder=r"Data/"

def get_file_path(file_type,stock_number,date):
    date_=date.strftime("%Y-%m-%d")
    filename=file_type+'_'+str(stock_number)+'_'+date_+".json"
    path=traget_floder+filename
    return path

def Reading_from_json_file(file_type,stock_number,date):
    file_path=get_file_path(file_type,stock_number,date)
    with open(file_path, 'r') as openfile:
        json_object = json.load(openfile)
    return json_object

def writing_to_json(file_type,stock_number,date,dictionary):
    file_path=get_file_path(file_type,stock_number,date)
    with open(file_path, "w") as outfile:
        json.dump(dictionary, outfile)

In [3]:
def get_DB():
    return pd.read_csv('BaseInfo/His_Stock.csv',dtype={"stock_number": "string","date": "string"})

In [33]:
def delete_DB_stock_by_Month_Stock_number(strMonth,stock_number,type_):
    df=get_DB()
    df_new = df[~((df['type'] == type_) & (df['date'] == strMonth) & (df['stock_number'] == stock_number))]
    df_new.to_csv('BaseInfo/His_Stock.csv',index=False) 

In [5]:
def delete_DB_stock_by_Month(strMonth):
    df=get_DB()
    df_new =df[df['date']!=strMonth]
    df_new.to_csv('BaseInfo/His_Stock.csv',index=False) 

# 公用 BaseＦＵＮＣ

In [6]:
# 確認是否已被抓取過
def check_data_exist(file_type,stock_number,date):
    #print("######## check_data_exist ######")
    #print("file_type =>",file_type)
    #print("stock_number =>",stock_number)
    #print("date =>",date)
    His_Stock=get_DB()
    stock_number_=str(stock_number)
    date_=date.strftime("%Y-%m-%d")
    
    check=His_Stock[(His_Stock['type']==file_type)]
    check=check[(check['stock_number']==stock_number_)]
    check=check[(check['date']==date_)]
    check=check[(check['status']=='OK')]
    if len(check.index)==0 :
        return False
    return True

def log_(His_Stock,file_type,stock_number,date,status,data):
    His_Stock=get_DB()
    df2=pd.DataFrame({
        "type":file_type,
        "stock_number":str(stock_number),
        "date":date.strftime("%Y-%m-%d"),
        "status":status,
    },index=[0]) 
    writing_to_json(file_type,stock_number,date,data)
    His_Stock=His_Stock.dropna(axis=0)
    df_log=His_Stock.append(df2, ignore_index=True)
    His_Stock=df_log.drop_duplicates()
    His_Stock=His_Stock.sort_values(['stock_number','type'],ascending=True).reset_index(drop=True)
    His_Stock.to_csv('BaseInfo/His_Stock.csv',index=False)   
    return df_log

In [7]:
def  GetStockInfoByID(StocKID):
    data=StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv', converters={'公司代號': str})
    result=data[data['公司代號']==StocKID]
    if result.iloc[0]['Type']=="上櫃" :
        return result.iloc[0]['公司簡稱']+'-'+result.iloc[0]['上櫃公司產業類別']+'-'+result.iloc[0]['Type']
    else:
        return result.iloc[0]['公司簡稱']+'-'+result.iloc[0]['上市公司產業類別']+'-'+result.iloc[0]['Type']

    #return result.iloc[0]['公司簡稱']+'-'+result.iloc[0]['上市公司產業類別']

In [8]:
def  GetStockTypeByID(StocKID):
    data=StockBaseData=pd.read_csv('BaseInfo/StockBaseData.csv', converters={'公司代號': str})
    result=data[data['公司代號']==StocKID]
    return result.iloc[0]['Type']

# 上市公司 # 爬取每月股價

爬取每月股價的目標網站並包裝成函式

UI https://www.twse.com.tw/zh/trading/historical/stock-day.html

In [9]:
def craw_one_month(file_type,stock_number,date):
    His_Stock=get_DB()
    url=''
    if GetStockTypeByID(str(stock_number))=='上市':
        url = (
            "http://www.twse.com.tw/exchangeReport/STOCK_DAY?response=json&date="+
            date.strftime('%Y%m%d')+
            "&stockNo="+
            str(stock_number)
        )
        data = json.loads(urlopen(url).read())
    else :
        url=("https://www.tpex.org.tw/web/stock/aftertrading/daily_trading_info/st43_result.php?l=zh-tw&d="+
             
        str(int(date.strftime('%Y'))-1911)+
         date.strftime('/%m/%d')+
         "&stkno="+
          str(stock_number)
        )
        data = json.loads(urlopen(url).read())
        data['fields']=['日期','成交股數','成交金額','開盤價','最高價','最低價','收盤價','漲跌價差','成交筆數']
        data['data']=data['aaData']
        data['stat']='OK'
        
    # For log
    status=data['stat']
    His_Stock=log_(His_Stock,file_type,stock_number,date,status,data)

    if status=="OK":
        return pd.DataFrame(data['data'],columns=data['fields']),His_Stock

    return pd.DataFrame(),His_Stock



#根據使用者輸入的日期，以月為單位，重複呼叫爬取月股價的函式
#https://ithelp.ithome.com.tw/articles/10213955
def craw_stock(stock_number, start_month,end_time=datetime.now().strftime("%Y-%m-%d") ):
    His_Stock=get_DB()
    file_type="Stock_His"
    b_month = date(*[int(x) for x in start_month.split('-')])
    e_month = date(*[int(x) for x in end_time.split('-')])
    result = pd.DataFrame()
    isSuccess=False
    for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
        try: 
            if check_data_exist(file_type,stock_number,dt):
                #內部存取
                #print(file_type,stock_number,dt.strftime("%Y-%m-%d"),"===>內部存取")
                data=Reading_from_json_file(file_type,stock_number,dt)
                get_df_stock_data=pd.DataFrame(data['data'],columns=data['fields'])
                isSuccess=True
            else:
                #外部抓取
                print(file_type,stock_number,dt.strftime("%Y-%m-%d"),"===>外部抓取")
                get_df_stock_data,His_Stock=craw_one_month(file_type,stock_number,dt)
                time.sleep(8) #如果在 6 秒內連續擷取兩次，那 IP 就會被鎖住，要一個小時後才能繼續擷取
                isSuccess=True
            result = pd.concat([result,get_df_stock_data],ignore_index=True)   
        except Exception as error:
            print('【_CrawStock】 craw_stock error -->'+file_type,stock_number,dt.strftime("%Y-%m-%d") )
            print("An exception occurred:", error)
    return result,His_Stock,isSuccess



In [10]:
def DataCleaning(input_RowData):    
    
    # 如果不使用copy 會導致變動到原本資料  https://www.796t.com/article.php?id=22560
    df=input_RowData.copy()
    #處理資料格式 
    df['Year']=df['日期'].str.split('/', 1).str[0]
    df['Date']=df['日期'].str.split('/', 1).str[1]
    df['Year']=df.apply(lambda x:int( x['Year'])+1911 , axis=1)
    df['日期']=df.apply(lambda x: str(x['Year'])+'/'+ x['Date'], axis=1)
    df['Date_']= df['日期']
    df['日期']=df.apply(lambda x: x['日期'].replace("＊",""), axis=1)
  
    df.set_index("日期", inplace=True)
    df['成交股數']=df.apply(lambda x: x['成交股數'].replace(",",""), axis=1)
    df['成交股數'] = df['成交股數'].astype(int)
    df['成交筆數']=df.apply(lambda x: x['成交筆數'].replace(",",""), axis=1)
    df['成交筆數'] = df['成交筆數'].astype(int)

    df = df[['開盤價','最高價','最低價','收盤價','成交筆數']]
    
    df.index.name = '日期'
    df.index = pd.to_datetime(df.index)
    #df = df.astype(float)


    df=df[df["開盤價"]!='--']
    df["開盤價"] = df["開盤價"].str.replace(",","").astype(float)

    df["最高價"] = df["最高價"].str.replace(",", "").astype(float)
    df["最低價"] = df["最低價"].str.replace(",", "").astype(float)
    df["收盤價"] = df["收盤價"].str.replace(",", "").astype(float)
    df["成交筆數"] = df["成交筆數"].astype(float)
    
    df_ohlcv = {'開盤價': 'first',
               '最高價': 'max',
               '最低價': 'min',
               '收盤價': 'last',
               '成交筆數': 'sum',}


    df_final = df.resample('D',closed='left', label='left').agg(df_ohlcv).copy()
    df_final.index = mdates.date2num(df_final.index)
    
    data_final = df_final.reset_index().values
    
    return  data_final,df_final

#  Revenue

In [11]:

def crawdata(stock_number):
    His_Stock=get_DB()
    file_type="Revenue_His"
    #外部抓取
    url = "https://stock.wearn.com/asale2.asp?kind="+stock_number

    response = requests.get(url,verify = False)
    response.encoding = 'big5'
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, "html.parser")
        table=soup.find('table')
        columns = [th.text.replace('\n', '') for th in table.find_all('tr')[2].find_all('th')]
        columns.insert(0, "month")
        trs = table.find_all('tr')[3:]
        rows = list()
        for tr in trs:
            rows.append([td.text.replace('\n', '').replace('\xa0', '') for td in tr.find_all('td')])
        rows[:5]
        df = pd.DataFrame(data=rows, columns=columns)
        print(df)
        df=df[(df['當月營收'] !='')]
        df['當月營收']=df['當月營收'].str.replace(',', '').astype(float)
        new =df['month'].str.split('/', 1, expand = True)
        df['Year']=new[0].astype(int)+1911
        df['M']=new[1].astype(int)
        df['dt']=pd.to_datetime(df['Year'].astype(str)+'/'+df['M'].astype(str)+'/01')
        df['Date']=pd.to_datetime(df['Year'].astype(str)+'/'+df['M'].astype(str)+'/01')
        df = df.set_index('dt')
        df=df.sort_index()
        df['公司代號']=stock_number
        statusrevenue='OK'
        dfrevenue=df
    else:
        statusrevenue='網站無法訪問'
        

    # for log
    if  statusrevenue=='OK':
        df_tojson=dfrevenue.copy()
        df_tojson.reset_index(drop=True, inplace=True)
        data = df_tojson.to_json(orient="split", force_ascii=False) 
        status="OK"
        date=dfrevenue.sort_index(ascending=False)[0:1].index[0]
        His_Stock=log_(His_Stock,file_type,stock_number,date,status,data)
    return dfrevenue,statusrevenue


def craw_revenue(stock_number):
    His_Stock=get_DB()
    file_type="Revenue_His"
    #Setting
    craw=False #確定是否對外爬取的FLAG
    checkData=His_Stock.loc[(His_Stock['stock_number'] == stock_number ) & (His_Stock['type'] == file_type)]
  
    if checkData.shape[0]==0 :
        #DB　沒資料
        dtdate=datetime.now()
        craw=True
    else :
        #DB 有資料，判斷資料是否過舊
        
        #最後取到資料營收的年月 EX '2024-06-01'
        logdatetime_str =checkData.sort_index(ascending=False).iloc[0].date
        dtdate=datetime.strptime(logdatetime_str, '%Y-%m-%d')
        
        #若要更新7月的資訊 大約在8/10號去更新  (兩個月+10天)
        need_update_date=datetime.strptime(logdatetime_str, '%Y-%m-%d') + relativedelta(months=2) + relativedelta(days=9)
        
        # revenue大概更新日在隔月中
        if datetime.now()>need_update_date:   
            craw=True

    if check_data_exist(file_type,stock_number,dtdate) and craw==False:
        #內部讀取
        #print(dtdate,"#內部讀取")
        data=Reading_from_json_file(file_type,stock_number,dtdate)
        xxx=json.loads(data)
        revenue_data=pd.DataFrame(xxx['data'],columns=xxx['columns'])
        df=revenue_data
    else:
        #外部抓取
        print(dtdate,"#外部抓取")
        df,status=crawdata(stock_number)
        time.sleep(3);

    return df,His_Stock

def RevenueDataCleaning(input_RowData):
  
    rowdata_revenue=input_RowData.copy()
    rowdata_revenue['Date']=pd.to_datetime(rowdata_revenue['Year'].astype(str)+'/'+rowdata_revenue['M'].astype(str)+'/01')
    rowdata_revenue['revenue']=rowdata_revenue['當月營收']
    revenue_df = rowdata_revenue[['Date','revenue']]
    revenue_df.set_index("Date", inplace=True)
    revenue_df.index.name = 'date'
    revenue_df.index = pd.to_datetime(revenue_df.index)
    return revenue_df


start_month='2023-01-01'
end_time='2024-07-22'
stock_number='1101'

RowData_df_craw_stock,His_Stock = craw_stock(stock_number,start_month,end_time)
data_final,df_final=DataCleaning(RowData_df_craw_stock)

RowData_df_craw_revenue,His_Stock=craw_revenue('1101')
df_Revenue=RevenueDataCleaning(RowData_df_craw_revenue)

df_Revenue

start_month='2023-01-01'
end_time='2024-07-22'
stock_number='3693'

b_month = date(*[int(x) for x in start_month.split('-')])
e_month = date(*[int(x) for x in end_time.split('-')])
result = pd.DataFrame()
    
for dt in rrule.rrule(rrule.MONTHLY, dtstart=b_month, until=e_month):
  
    print(dt,check_data_exist('Stock_His',stock_number,dt))